In [ ]:
from libs import *
from TSPex import *
from TSPnna import *

In [ ]:
df_vtx = pd.read_excel('data/vertices.xlsx')
vtx_list = []
# Create the list from the excel filde and stor it as list of objects
for i in range(len(df_vtx)):
    vtx = Vtx(df_vtx['Ver_Name'][i], df_vtx['x'][i], df_vtx['y'][i])
    vtx_list.append(vtx)

In [ ]:
vtx_list 

In [ ]:
obstacles_data = [  
    (1, 0, 2, 0, "T"),  
    (3, 0, 4, 0, "T"),  
    (2, 1, 3, 1, "T"),  
    (1, 1, 1, 2, "N"),  
    (1, 2, 2, 2, "N"),  
    (4, 4, 5, 4, "B"),  
    (5, 4, 5, 5, "B")  
]  

# Test Set Up code - Distance

In [ ]:
board = Board(obstacles_data) 
start = vtx_list[0]
end = vtx_list[1]

number_of_paths = vtx_list[0].optimized_paths(vtx_list[1])

initial_dir = (0,1)
min_path, min_dis, min_turn_guidance, min_last_direction = board.find_optimized_paths(initial_dir, start, end, number_of_paths)

print(f'The suitable path is {min_path}')
print(f'The suitable distance is {min_dis}')
print(f'The guidance: {min_turn_guidance}')
print(f'The last direction: {min_last_direction}')

# Test Exhaustive search Algo

In [ ]:
board = Board(obstacles_data)  
initial_dir = (1,0)  
min_result = min_possible_path(board, vtx_list, initial_dir)

In [ ]:
print(f"Path : {min_result['Path']}")  
print(f"  Total Distance: {min_result['Total_Distance']}")  
print(f"  Guidance:")  
for guidance in min_result['Guidance']:  
    print(f"    - {guidance}")  
print(f"  Guided_Path:")
for guidance in min_result['Guided_Path']:
    print(f"    - {guidance}")
print()

# Test Nearest Neighbourhood Algo

In [ ]:
board = Board(obstacles_data)  
initial_dir = (1, 0)  
min_result = nearest_neighbor_path(board, vtx_list, initial_dir)  

In [ ]:
print('The optimal guided_path given by the nearest neighbor algorithm is')  
print(f"Path : {min_result['Path']}")  
print(f"  Total Distance: {min_result['Total_Distance']}")  
print(f"  Guidance:")  
for guidance in min_result['Guidance']:  
    print(f"    - {guidance}")  
print(f"  Guided_Path:")  
for guidance in min_result['Guided_Path']:  
    print(f"    - {guidance}")  
print()